In [1]:
# Import dependencies
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# Read in csv files
csv_big_dance = "Resources/big_dance_csv (1).csv"
csv_draft_pick = "Resources/1976_to_2015_draftees_edit2 (1).csv"

In [3]:
# Create pandas table for tournament data
tourney_data = pd.read_csv(csv_big_dance)
tourney_data.head()

,Year,Round,Region Number,Region Name,Seed,Score,Team,Team.1,Score.1,Seed.1
0,1985,1,1,West,1,83,St Johns,Southern,59,16
1,1985,1,1,West,2,81,VCU,Marshall,65,15
2,1985,1,1,West,3,65,NC State,Nevada,56,14
3,1985,1,1,West,4,85,UNLV,San Diego St,80,13
4,1985,1,1,West,5,58,Washington,Kentucky,65,12


In [4]:
# Drop some columns that are not needed and rename columns so they match in SQL database
tourney_data = tourney_data[["Year", "Round", "Seed", "Team", "Team.1", "Seed.1"]]
tourney_data_df = tourney_data.rename(columns={"Year":"year", "Round":"round", "Seed":"seed_1", "Team":"college_1", "Team.1":"college_2", "Seed.1":"seed_2"})
tourney_data_df.head()

,year,round,seed_1,college_1,college_2,seed_2
0,1985,1,1,St Johns,Southern,16
1,1985,1,2,VCU,Marshall,15
2,1985,1,3,NC State,Nevada,14
3,1985,1,4,UNLV,San Diego St,13
4,1985,1,5,Washington,Kentucky,12


In [5]:
tourney_data_clean = tourney_data_df.loc[tourney_data_df["year"] != 2016]
tourney_data_clean = tourney_data_clean.loc[tourney_data_df["year"] != 2017]
tourney_data_clean = tourney_data_clean.loc[tourney_data_df["year"] != 2018]
tourney_data_cleaned = tourney_data_clean.loc[tourney_data_df["year"] != 2019]
tourney_data_cleaned

,year,round,seed_1,college_1,college_2,seed_2
0,1985,1,1,St Johns,Southern,16
1,1985,1,2,VCU,Marshall,15
2,1985,1,3,NC State,Nevada,14
3,1985,1,4,UNLV,San Diego St,13
4,1985,1,5,Washington,Kentucky,12
...,...,...,...,...,...,...
1948,2015,4,4,Louisville,Michigan St,7
1949,2015,4,1,Duke,Gonzaga,2
1950,2015,5,1,Kentucky,Wisconsin,1
1951,2015,5,7,Michigan St,Duke,1


In [6]:
# Create pandas table for draft pick data
draft_pick = pd.read_csv(csv_draft_pick)
draft_pick.head()

,Unnamed: 0,Player,All_NBA,All.Star,Draft_Yr,Pk,Team,College,Yrs,Games,...,Executive,Tenure,Exec_ID,Exec_draft_exp,attend_college,first_year,second_year,third_year,fourth_year,fifth_year
0,1,Robert Parish,2,9,1976,8,GSW,Centenary College of Louisiana,21,1611,...,Al Attles,3641 days 00:00:00.000000000,1,1,1,0,0,0,0,0
1,2,Sonny Parker,0,0,1976,17,GSW,Texas A&M University,6,452,...,Al Attles,3641 days 00:00:00.000000000,1,1,1,0,0,0,0,0
2,3,Marshall Rogers,0,0,1976,34,GSW,University of Texas-Pan American,1,26,...,Al Attles,3641 days 00:00:00.000000000,1,1,1,0,0,0,0,0
3,4,Jeff Fosnes,0,0,1976,68,GSW,Vanderbilt University,0,0,...,Al Attles,3641 days 00:00:00.000000000,1,1,1,0,0,0,0,0
4,5,Carl Bird,0,0,1976,86,GSW,University of California,0,0,...,Al Attles,3641 days 00:00:00.000000000,1,1,1,0,0,0,0,0


In [7]:
# Drop some columns that are not needed and rename so it matches in SQL database
draft_pick = draft_pick[["Player", "Draft_Yr", "College"]]
draft_pick_df = draft_pick.rename(columns={"Player":"player", "Draft_Yr":"draft_yr", "College":"college"})
draft_pick_df.head()

,player,draft_yr,college
0,Robert Parish,1976,Centenary College of Louisiana
1,Sonny Parker,1976,Texas A&M University
2,Marshall Rogers,1976,University of Texas-Pan American
3,Jeff Fosnes,1976,Vanderbilt University
4,Carl Bird,1976,University of California


In [9]:
draft_pick_clean = draft_pick_df.loc[draft_pick_df["draft_yr"] != 1976]
draft_pick_clean = draft_pick_clean.loc[draft_pick_clean["draft_yr"] != 1977]
draft_pick_clean = draft_pick_clean.loc[draft_pick_clean["draft_yr"] != 1978]
draft_pick_clean = draft_pick_clean.loc[draft_pick_clean["draft_yr"] != 1979]
draft_pick_clean = draft_pick_clean.loc[draft_pick_clean["draft_yr"] != 1980]
draft_pick_clean = draft_pick_clean.loc[draft_pick_clean["draft_yr"] != 1981]
draft_pick_clean = draft_pick_clean.loc[draft_pick_clean["draft_yr"] != 1982]
draft_pick_clean = draft_pick_clean.loc[draft_pick_clean["draft_yr"] != 1983]
draft_pick_cleaned = draft_pick_clean.loc[draft_pick_clean["draft_yr"] != 1984]
draft_pick_cleaned

,player,draft_yr,college
96,Chris Mullin,1985,St. John's University
97,Brad Wright,1985,"University of California, Los Angeles"
98,Luster Goodwin,1985,University of Texas at El Paso
99,Greg Cavener,1985,University of Missouri
100,Gerald Crosby,1985,University of Georgia
...,...,...,...
3919,Steve Rogers,1992,Alabama State University
3920,Rex Walters,1993,University of Kansas
3921,John Best,1993,Tennessee Technological University
3922,Yinka Dare,1994,George Washington University


In [10]:
# Create the connection between pandas data frames and SQL database tables
connection_string = "postgres:101219@localhost:5432/ETL_project"
engine = create_engine(f'postgresql://{connection_string}')

In [11]:
# Check to see that the connection was made
engine.table_names()

['tournament_games', 'draft_picks']

In [12]:
# Upload the data frame tourney_data_df into SQL database table tournament_games
tourney_data_cleaned.to_sql(name='tournament_games', con=engine, if_exists='append', index=False)

In [13]:
# Upload the data frame draft_pick_df into SQL database table draft_picks
draft_pick_cleaned.to_sql(name='draft_picks', con=engine, if_exists='append', index=False)